<a href="https://colab.research.google.com/github/yakovsushenok/Thesis/blob/main/BirdSongIdentification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import zipfile
import os
import pandas as pd
import math, random
import torch
import torchaudio
from torchaudio import transforms
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import init
from google.colab import drive
from sklearn.model_selection import StratifiedShuffleSplit
drive.mount('/content/gdrive')
import time
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib
try:
    from scipy.fftpack import fft, ifft
except ImportError:
    from numpy.fft import fft, ifft
from scipy.signal import lfilter
import scipy.io as sio
from scipy import signal
import gc
import h5py
random.seed(0)
np.random.seed(0)

Mounted at /content/gdrive


# Data

In [2]:
# extracting the training data (audio files) from the zip file (12 minutes)
zip_ref = zipfile.ZipFile('/content/gdrive/MyDrive/train_short_audio.zip', 'r')
zip_ref.extractall('/content/tmp') 
zip_ref.close()

In [36]:
# extracting the data which has audio files that are of similar length to the testing data
zip_ref = zipfile.ZipFile('/content/gdrive/MyDrive/train_soundscapes.zip', 'r')
zip_ref.extractall('/content/trainSoundscapes') 
zip_ref.close()

df = pd.read_csv("/content/gdrive/MyDrive/train_metadata.csv") # the metadata

# The dataset below is the subsetted dataset with labels that have 500 occurrences
#df = pd.read_csv("/content/gdrive/MyDrive/train_metadata_more_than_500.csv")
df['relative_path'] = '/content/tmp/' + df['primary_label'] + '/' + df['filename'] 

### Subsetting the data so that we're left with subspecies which have 300+ samples and with audio rated 4.0+

In [37]:
df = df[(df['primary_label'].value_counts().reindex(df['primary_label'])>299).values & (df['rating'] > 3.5)]
df = df[['relative_path', 'primary_label']]
unique_labels = df.primary_label.unique()
mapping = dict(zip(unique_labels, range(len(unique_labels))))
df.primary_label = df.primary_label.map(mapping)

### We're going to partition our data into a training/validation/testing test with 80% being the training, 10% for validating and 10% for testing with each species category having the same distribution.

In [39]:
# splitting into train, val+test
X, y = df['relative_path'], df['primary_label']
split1 = StratifiedShuffleSplit(n_splits=2, test_size=0.2, random_state=0)
train_index, val_test = next(split1.split(X, y))
# splitting into val, test
X1, y1 = df.iloc[val_test, 0], df.iloc[val_test, 1]
split2 = StratifiedShuffleSplit(n_splits=2, test_size=0.5, random_state=0)
val_index, test_index = next(split2.split(X1, y1))
# subsetting the datasets
df_train = df.iloc[train_index,:]
df_val = df.iloc[val_index, :]
df_test = df.iloc[test_index, :]

# Utility classes

In [5]:
### MRCG CODE
epsc = 0.000001

def mrcg_extract(sig, sampFreq = 32000): # Sample frequency is always 32,000 in our case
    # Code From: https://github.com/MoongMoong/MRCG_python/blob/master/MRCG_python_master/mrcg/MRCG.py
    
    beta = 1000 / np.sqrt(sum(map(lambda x:x*x,sig)) / len(sig))
    sig = sig*beta
    sig = sig.reshape(len(sig), 1)
    g = gammatone(sig, 64, sampFreq)
    cochlea1 = np.log10(cochleagram(g, int(sampFreq * 0.025), int(sampFreq * 0.010)) + 0.0000005)
    cochlea2 = np.log10(cochleagram(g, int(sampFreq * 0.200), int(sampFreq * 0.010)) + 0.0000005) # 768, x 
    cochlea1 = cochlea1[:,:]
    cochlea2 = cochlea2[:,:]
    cochlea3 = get_avg(cochlea1, 5, 5)
    cochlea4 = get_avg(cochlea1, 11, 11)
    
    all_cochleas = np.concatenate([cochlea1,cochlea2,cochlea3,cochlea4],0)
    del0 = deltas(all_cochleas)
    ddel = deltas(deltas(all_cochleas, 5), 5)

    ouotput = np.concatenate((all_cochleas, del0, ddel), 0)

    return ouotput

def gammatone(insig, numChan=128, fs = 16000): # 
    fRange = [1000, 20000] # try from 1000 to 20000 (was [50, 8000])
    filterOrder = 4
    gL = 2048
    sigLength = len(insig)
    phase = np.zeros([numChan, 1])
    erb_b = hz2erb(fRange)

    
    erb_b_diff = (erb_b[1]-erb_b[0])/(numChan-1)
    erb = np.arange(erb_b[0], erb_b[1]+epsc, erb_b_diff)
    cf = erb2hz(erb)
    b = [1.019 * 24.7 * (4.37 * x / 1000 + 1) for x in cf]
    gt = np.zeros([numChan, gL])
    tmp_t = np.arange(1,gL+1)/fs
    for i in range(numChan):
        gain = 10**((loudness(cf[i])-60)/20)/3*(2 * np.pi * b[i] / fs)**4
        tmp_temp = [gain*(fs**3)*x**(filterOrder - 1)*np.exp(-2 * np.pi * b[i] * x)*np.cos(2 * np.pi * cf[i] * x + phase[i]) for x in tmp_t]
        tmp_temp2 = np.reshape(tmp_temp, [1, gL])

        gt[i, :] = tmp_temp2

    sig = np.reshape(insig,[sigLength,1])
    gt2 = np.transpose(gt)
    resig = np.matlib.repmat(sig,1,numChan)
    r = np.transpose(fftfilt(gt2,resig,numChan))
    return r

def hz2erb(hz):  
    erb1 = 0.00437
    erb2 = np.multiply(erb1,hz)
    erb3 = np.subtract(erb2,-1)
    erb4 = np.log10(erb3)
    erb = 21.4 *erb4
    return erb

def erb2hz(erb): 
    hz = [(10**(x/21.4)-1)/(0.00437) for x in erb]
    return hz

def loudness(freq): 
    dB=60
    fmat = sio.loadmat('/content/gdrive/MyDrive/f_af_bf_cf.mat')
    af = fmat['af'][0]
    bf = fmat['bf'][0]
    cf = fmat['cf'][0]
    ff = fmat['ff'][0]
    i = 0
    while ff[i] < freq and i < len(ff) - 1: # my code:  i < len(ff)
        i = i + 1

    afy = af[i - 1] + (freq - ff[i - 1]) * (af[i] - af[i - 1]) / (ff[i] - ff[i - 1])
    bfy = bf[i - 1] + (freq - ff[i - 1]) * (bf[i] - bf[i - 1]) / (ff[i] - ff[i - 1])
    cfy = cf[i - 1] + (freq - ff[i - 1]) * (cf[i] - cf[i - 1]) / (ff[i] - ff[i - 1])
    loud = 4.2 + afy * (dB - cfy) / (1 + bfy * (dB - cfy))
    return loud

def fftfilt(b,x,nfft): 
    fftflops = [18, 59, 138, 303, 660, 1441, 3150, 6875, 14952, 32373, 69762,
                149647, 319644, 680105, 1441974, 3047619, 6422736, 13500637, 28311786,
                59244791, 59244791*2.09]
    nb, _ = np.shape(b)
    nx, mx = np.shape(x)
    n_min = 0
    while 2**n_min < nb-1:
        n_min = n_min+1
    n_temp = np.arange(n_min, 21 + epsc, 1)
    n = np.power(2,n_temp)
    fftflops = fftflops[n_min-1:21]
    L = np.subtract(n,nb-1)
    lenL= np.size(L)
    temp_ind0 = np.ceil(np.divide(nx,L))
    temp_ind = np.multiply(temp_ind0,fftflops)
    temp_ind = np.array(temp_ind)
    ind = np.argmin(temp_ind)
    nfft=int(n[ind])
    L=int(L[ind])
    b_tr = np.transpose(b)
    B_tr = fft(b_tr,nfft)
    B = np.transpose(B_tr)
    y = np.zeros([nx, mx])
    istart = 0
    while istart < nx :
        iend = min(istart+L,nx)
        if (iend - istart) == 1 :
            X = x[0][0]*np.ones([nx,mx])
        else :
            xtr = np.transpose(x[istart:iend][:])
            Xtr = fft(xtr,nfft)
            X = np.transpose(Xtr)
        temp_Y = np.transpose(np.multiply(B,X))
        Ytr = ifft(temp_Y,nfft)
        Y = np.transpose(Ytr)
        yend = np.min([nx, istart + nfft])
        y[istart:yend][:] = y[istart:yend][:] + np.real(Y[0:yend-istart][:])

        istart = istart + L
    
    return y

def cochleagram(r, winLength = 320, winShift=160): 
    numChan, sigLength = np.shape(r)
    increment = winLength / winShift
    M = np.floor(sigLength / winShift)
    a = np.zeros([numChan, int(M)])
    rs = np.square(r)
    rsl = np.concatenate((np.zeros([numChan,winLength-winShift]),rs),1)
    for m in range(int(M)):
        temp = rsl[:,m*winShift : m*winShift+winLength]
        a[:, m] = np.sum(temp,1)

    return a

def get_avg( m , v_span, h_span): 
    nr,nc = np.shape(m)

    fil_size = (2 * v_span + 1) * (2 * h_span + 1)
    meanfil = np.ones([1+2*h_span,1+2*h_span])
    meanfil = np.divide(meanfil,fil_size)

    out = signal.convolve2d(m, meanfil, boundary='fill', fillvalue=0, mode='same')
    return out

def deltas(x, w=9) : 
    nr,nc = np.shape(x)
    if nc ==0 :
        d= x
    else :
        hlen = int(np.floor(w / 2))
        w = 2 * hlen + 1
        win=np.arange(hlen, int(-(hlen+1)), -1)
        temp = x[:, 0]
        fx = np.matlib.repmat(temp.reshape([-1,1]), 1, int(hlen))
        temp = x[:, nc-1]
        ex = np.matlib.repmat(temp.reshape([-1,1]), 1, int(hlen))
        xx = np.concatenate((fx, x, ex),1)
        d = lfilter(win, 1, xx, 1)
        d = d[:,2*hlen:nc+2*hlen]

    return d

Now that I have the MRCG code, I want to create a dataset which will hold these MRCG values. This will mean I'll not have to pre-process them every run of the model.

In [6]:
def get_mrcg_from_file(file):
  
  mid = 650000 # zero padding so that mrcgs are 20 seconds max
  waveform, sr = torchaudio.load(file)
  if len(waveform[0]) < mid:
    target = torch.zeros(mid)
    source = waveform[0]
    target[:len(source)] = source
    return mrcg_extract(target)
  else:
    waveform = waveform[0]
    return mrcg_extract(waveform[:mid])

In [ ]:
df_list = [i for i in range(0,8600, 100)]
df_list[-1] = len(df_train)

for i in range(67,len(df_list)-1):    # [2706, 0950, stopped at 22] <-- missed 22 (i=21+1)... need to do it.
                                   # [2806, 2906, stopped at i=43+1]
                                   # for some reason, indices i=14-19 are not being processed (for the tensors)
  
  
  
  
                                   # until 2906, 1920 I have been creating df_train_tensor's with the index i in the get_mrcg_from_file(df['relative_path'].iloc[j]) thing. Wasted 2-3 days on this. 
                                   # On 2906 I'm creating a dataset with fRange = [1000, 20000] as per my supervisors' 
  
  print(f"currently on df {i+1}")
  df = df_train[df_list[i]:df_list[i+1]]
  #df['mrcg'] = df['relative_path'].apply(get_mrcg_from_file)
  df.to_csv(f'/content/gdrive/MyDrive/df_train{i+1}.csv') 
  x = torch.tensor(get_mrcg_from_file(df['relative_path'].iloc[0]))     
  
  x = x[None, : , :]
  for j in range(1,len(df)):
    x1 = torch.tensor(get_mrcg_from_file(df['relative_path'].iloc[j])) # 12 sec per sample for mrcg
    x1 = x1[None, :, :]
    x = torch.cat((x,x1), 0)
  print(f"saving tensor {i+1}")
  torch.save(x, f'/content/gdrive/MyDrive/df_train_tensor{i+1}.pt')

# Creating a hp5y dataset and dataloader
https://discuss.pytorch.org/t/save-torch-tensors-as-hdf5/39556

In [ ]:
tensors = h5py.File('/content/gdrive/MyDrive/train20_tensors.h5', 'w') 

In [8]:
N = 8497
data_train_predictor = tensors.create_dataset('data', shape=(N, 768, 2031), dtype=np.float32, fillvalue=0)

In [9]:
ind_range = [i for i in range(0,8600,100)]
ind_range[-1]= 8497

In [10]:
for i in range(1,len(ind_range)):
    x = torch.load(f"/content/gdrive/MyDrive/df_train_tensor{i}.pt")
    data_train_predictor[ind_range[i-1]:ind_range[i]] = x

In [11]:
tensors.close()

In [5]:
class H5DS(Dataset):
  def __init__(self, df, path):
    self.path = path
    self.data = h5py.File(self.path, 'r') 
    self.data = self.data['data']
    self.df = df
  
  def __len__(self):
    return len(self.df)    
    
  def __getitem__(self, idx):
   
   return (self.data[idx], torch.tensor(self.df['primary_label'].iloc[idx]))

# Subset from 8497



In [43]:
tensors = h5py.File('/content/gdrive/MyDrive/train20_tensors.h5', 'r') 

In [45]:
randomlist = random.sample(range(8497), 2000)
randomlist.sort()

In [46]:
train20_tensors_subset = tensors['data'][randomlist]
ts = h5py.File('/content/gdrive/MyDrive/train20_tensors_subset.h5', 'w') 
data_train_predictor_subset = ts.create_dataset('data', shape=(2000, 768, 2031), dtype=np.float32, fillvalue=0)

In [47]:
data_train_predictor_subset[:] = train20_tensors_subset

In [48]:
ts = h5py.File('/content/gdrive/MyDrive/train20_tensors_subset.h5', 'r') 
tsd = ts['data']
tsd[0].shape

(768, 2031)

In [49]:
df_train_subset = df_train.iloc[randomlist, :]
len(df_train_subset['primary_label'].value_counts())

39

# Model

In [6]:
class AudioClassifier(nn.Module): # MLP
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.input_fc = nn.Linear(input_dim, 50)
        self.hidden_fc = nn.Linear(50, 50)
        self.output_fc = nn.Linear(50, output_dim)

    def forward(self, x):

        batch_size = x.shape[0]

        x = x.view(batch_size, -1)
    
        h_1 = F.relu(self.input_fc(x))

        h_2 = F.relu(self.hidden_fc(h_1))

        y_pred = self.output_fc(h_2)

        return y_pred


class AudioClassifier(nn.Module): # CNN1
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.conv_layer1 = nn.Conv1d(768, 500, 3, stride = 1)
        self.conv_layer2 = nn.Conv1d(500, 300, 3, stride = 1)
        self.conv_layer3 = nn.Conv1d(300, 100, 3, stride = 1)
        self.conv_layer4 = nn.Conv1d(100, 1, 3, stride = 1)
        self.output_fc = nn.Linear(2023, output_dim)

    def forward(self, x):

    
        h_1 = F.relu(self.conv_layer1(x))
        h_2 = F.relu(self.conv_layer2(h_1))
        h_3 = F.relu(self.conv_layer3(h_2))
        h_4 = F.relu(self.conv_layer4(h_3))

        y_pred = self.output_fc(h_4)

        return y_pred.squeeze(1)

In [41]:
model = AudioClassifier(768*2031, 39)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,772,137 trainable parameters


# Training

In [60]:
# Create the model and put it on the GPU if available
myModel = AudioClassifier(768*2031, 39)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #
myModel = myModel.to(device)

def training(model, train_dl, num_epochs, val_dl):
  # Loss Function, Optimizer 
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
  
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                steps_per_epoch=int(len(train_dl)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')

  # Epoch iterator
  for epoch in range(num_epochs):
    running_loss = 0.0
    correct_prediction = 0
    total_prediction = 0
    indices = []
    # Batch iterator
    for i, data in enumerate(train_dl):

        inputs, labels = torch.tensor(data[0]).to(device), torch.tensor(data[1]).to(device) # Get the input features and target labels, and put them on the GPU
        
        # Normalize the inputs
        # inputs_m, inputs_s = inputs.mean(), inputs.std()
        # inputs = (inputs - inputs_m) / inputs_s

        
        optimizer.zero_grad() # Zero the parameter gradients

        # forward + backward + optimize
        outputs = model(inputs.float())
      
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item() # Keep stats for Loss and Accuracy

        _, prediction = torch.max(outputs,1) # Get the predicted class with the highest score
        correct_prediction += (prediction == labels).sum().item() # Count of predictions that matched the target label
        total_prediction += prediction.shape[0]

    
    # Print stats at the end of the epoch
    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    acc = correct_prediction/total_prediction
    print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')
    print(f"TESTING:")
    inference(model, val_dl)
    print("\n")

  print('Finished Training')

path = "/content/gdrive/MyDrive/train20_tensors.h5"
NUM_EPOCHS = 15
# Initializing the dataset
myds = H5DS(df_train, path)
# Random split of 80:20 between training and validation
num_items = len(myds)
num_train = round(num_items*(0.99))
num_val = num_items - num_train
print(num_train,num_val)
train_ds, val_ds = random_split(myds, [num_train, num_val])
# Create training and validation data loaders
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=10, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=10, shuffle=False)

training(myModel, train_dl, NUM_EPOCHS, val_dl) # Training

8412 85


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch: 0, Loss: 3.56, Accuracy: 0.06
TESTING:
Val Accuracy: 0.11


Epoch: 1, Loss: 3.27, Accuracy: 0.12
TESTING:
Val Accuracy: 0.09


Epoch: 2, Loss: 2.94, Accuracy: 0.20
TESTING:
Val Accuracy: 0.09


Epoch: 3, Loss: 2.51, Accuracy: 0.31
TESTING:
Val Accuracy: 0.05


Epoch: 4, Loss: 2.05, Accuracy: 0.43
TESTING:
Val Accuracy: 0.06


Epoch: 5, Loss: 1.51, Accuracy: 0.58
TESTING:
Val Accuracy: 0.05


Epoch: 6, Loss: 1.07, Accuracy: 0.70
TESTING:
Val Accuracy: 0.06


Epoch: 7, Loss: 0.79, Accuracy: 0.78
TESTING:
Val Accuracy: 0.04


Epoch: 8, Loss: 0.58, Accuracy: 0.84
TESTING:
Val Accuracy: 0.04


Epoch: 9, Loss: 0.42, Accuracy: 0.88
TESTING:
Val Accuracy: 0.05


Epoch: 10, Loss: 0.32, Accuracy: 0.92
TESTING:
Val Accuracy: 0.05


Epoch: 11, Loss: 0.24, Accuracy: 0.94
TESTING:
Val Accuracy: 0.05


Epoch: 12, Loss: 0.18, Accuracy: 0.95
TESTING:
Val Accuracy: 0.06


Epoch: 13, Loss: 0.16, Accuracy: 0.96
TESTING:
Val Accuracy: 0.07


Epoch: 14, Loss: 0.14, Accuracy: 0.96
TESTING:
Val Accurac

# Results (10 epochs):

---

Sample Length (s) : 20

Model: MLP

Normalization: Yes

Number of classes: 39

Mini-batch size = 1

Number of Samples in training: 8500*(0.8)

`fs = [1000,20000]`

Accurary = 0.05

Loss = 3.59

Input shape: `(768, 2031)`

Time taken for training: ~1h 30m

Trainable params: 77,994,989

---


Sample Length (s) : 20

Model: MLP 

Normalization: No

Number of classes: 39

Mini-batch size = 10

Number of Samples in training: 8500*(0.99)

`fs = [1000,20000]`

Input shape: `(768, 2031)`

Accurary = 0.05

Loss = 3.59

Time taken for training: 1:30 

Trainable params: 77,994,989

---

Sample Length (s) : 20

Model: CNN1 

Normalization: No

Number of classes: 39

Mini-batch size = 10

Number of Samples in training: 8500*(0.99)

`fs = [1000,20000]`

Input shape: `(768, 2031)`

Val Accurary = 0.10

Loss = N/A 

Time taken for training: 1:30 

Trainable params: 1,772,137

---







---

Epochs: 15

Sample Length (s) : 20

Model: CNN1 

Normalization: No

Number of classes: 39

Mini-batch size = 10

Number of Samples in training: 2000*(0.85)

`fs = [1000,20000]`

Input shape: `(768, 2031)`

Val Accurary = 0.05

Loss = N/A 

Time taken for training: 1:30 

Trainable params: 1,772,137

---

CURRENT:

Epochs: 15

Sample Length (s) : 20

Model: CNN1 

Normalization: Yes

Number of classes: 39

Mini-batch size = 10

Number of Samples in training: 2000*(0.85)

`fs = [1000,20000]`

Input shape: `(768, 2031)`

Val Accurary = 0.04 

Loss = N/A 

Time taken for training: 1:30 

Trainable params: 1,772,137

---


In [20]:
MyModel = None
torch.cuda.empty_cache()
with torch.no_grad():
    torch.cuda.empty_cache()

# Testing

In [56]:
def inference(model, val_dl):
  correct_prediction = 0
  total_prediction = 0

  # Disable gradient updates
  with torch.no_grad():
    for data in val_dl:
      # Get the input features and target labels, and put them on the GPU
      inputs, labels = data[0].to(device), data[1].to(device)

      # Normalize the inputs
      # inputs_m, inputs_s = inputs.mean(), inputs.std()
      # inputs = (inputs - inputs_m) / inputs_s

      # Get predictions
      outputs = model(inputs.float())

      # Get the predicted class with the highest score
      _, prediction = torch.max(outputs,1)
      # Count of predictions that matched the target label
      correct_prediction += (prediction == labels).sum().item()
      total_prediction += prediction.shape[0]
    
  acc = correct_prediction/total_prediction
  print(f'Val Accuracy: {acc:.2f}')

In [ ]:
inference(myModel, val_dl)